In [11]:
import numpy as np
from itertools import combinations
import random


In [12]:
INFINITE = 100000000
POPSIZE = 100

In [13]:
class Input:
    def __init__(self):
        self.G = []
        self.N = self.D = self.s = self.t = -1
        self.best = INFINITE
    def buildGraph(self, path):
        self.best = INFINITE
        with open(path) as f:
            line1 = f.readline()
            self.N, self.D = (int(i) for i in line1.split(" "))
            line2 = f.readline()
            self.s, self.t = (int(i) for i in line2.split(" "))
            self.s -= 1
            self.t -= 1
            edges = f.readlines()
            for node in range(self.N):
                temp = []
                for domain in range(self.D):
                    temp.append([])
                self.G.append(temp)
            
            index = 0
            modify = dict()
            for line in edges:
                index += 1
                u, v, w, d = (int(i) for i in line.split(" "))
                u -= 1
                v -= 1
                d -= 1
                if (u,v,d) in modify.keys():
                    modify[(u,v,d)] = min(modify[(u,v,d)], w)
                else:
                    modify[(u,v,d)] = w
            for key in modify.keys():
                u, v, d = key
                w = modify[key]
                self.G[u][d].append((w,v))
            return self.G
# t = Input()
# t.buildGraph('../Data/ex2.txt')
# r = set()
# kkk = 0
# for u in range(t.N):
#     for i in range(t.D):
#         for aa in t.G[u][i]: 
#             w,v = aa
#             if w < 46:
#                 print (f"{u} -> {v} in domain {i} : {w}")

In [14]:
x = np.random.rand(5)
print(x)


[0.98734445 0.29387543 0.04574738 0.90198078 0.88622272]


In [15]:
y = np.argsort(x)
print(y)

[2 1 4 3 0]


In [16]:
class Individual:
    def __init__(self, dataPath, randomKey):
        self.input = Input()
        self.input.buildGraph(dataPath)
        self.fitness = INFINITE
        self.randomKey = randomKey
        self.genes = np.argsort(randomKey)
    
    def eval(self):
        sizeGenes = len(self.genes)
        traversedDomain = []          
        currentDistance = np.full(self.input.N, INFINITE)
        currentDistance[self.input.s] = 0
        currentVertice = []
        currentVertice.append(self.input.s)


        for i in range(sizeGenes):
            isConnect = np.zeros(sizeGenes)
            for v in currentVertice:
                for j in range(sizeGenes):
                    if self.input.G[v][j]:
                        isConnect[j] = 1
                        
            c = -1
            for j in range(sizeGenes):
                if isConnect[self.genes[j]] > 0 and self.genes[j] not in traversedDomain:
                    c = self.genes[j]
            if c < 0:
                # print("c less than zero")
                return INFINITE
            traversedDomain.append(c)
            
            nextDistance = np.full(self.input.N, INFINITE)
            nextVertice = []
            check = np.zeros(self.input.N)
            for v in currentVertice:
                adjacentDistance = np.full(self.input.N, INFINITE)
                adjacentDistance[v] = 0
                for w,j in self.input.G[v][c]:
                    adjacentDistance[j] = w
                for j in range(self.input.N):
                    if adjacentDistance[j] != INFINITE:
                        if check[j] < 1: 
                            nextVertice.append(j)
                            check[j] = 1
                        nextDistance[j] = min(nextDistance[j], adjacentDistance[j] + currentDistance[v])
            currentVertice.clear()

            currentVertice += nextVertice
            for j in range(self.input.N):
                currentDistance[j] = nextDistance[j]
            self.fitness = min(self.fitness, currentDistance[self.input.t])
        return self.fitness
        
        

In [17]:
class LinkageTreeAlgo:
    def __init__(self, popSize, dataPath, numGeneration):
        self.popSize = popSize
        self.dataPath = dataPath
        self.input = Input()
        self.input.buildGraph(dataPath)
        self.pop = None
        self.cost = np.full(popSize, INFINITE)
        self.numGeneration = numGeneration 
        self.numEval = 0 
    def initPop(self):
        self.pop = np.array([Individual(self.dataPath, np.random.rand(self.input.D)) for i in range(self.popSize)])    
    def evalPop(self):
        self.numEval += self.popSize
        for i in range(self.popSize):
            self.cost[i] = self.pop[i].eval()
    def getBestCost(self):
        return np.min(self.cost)

    def adjacencyInfo(self, c1, c2, aIlookup):
        if (c1,c2) in aIlookup:
            return aIlookup[c1,c2]
        else:
            value = 0
            for i in range(self.popSize): 
                value += (self.pop[i].randomKey[c1] - self.pop[i].randomKey[c2])**2
            value = 1 - value/self.popSize
            aIlookup[c1,c2] = value
            return value
    def crossEntropy(self, c1, c2, cElookup):
        if(c1,c2) in cElookup:
            return cElookup[c1,c2]
        else:
            prob = 0
            for i in range(self.popSize):
                if self.pop[i].randomKey[c1] < self.pop[i].randomKey[c2]:
                    prob += 1
            prob /= self.popSize
            if prob == 0 or prob == 1:
                value = 0
            else:
                value = 1 + (prob * np.log2(prob) + (1-prob) * np.log2(1-prob))
            cElookup[c1, c2] = value
            return value
    def clusterDistance(self, c1, c2, cElookup, aIlookup, dislookup):
        if (c1, c2) in dislookup:
            return dislookup[c1, c2]
        else:
            value = self.adjacencyInfo(c1, c2, aIlookup) * self.crossEntropy(c1, c2, cElookup)
            dislookup[c1, c2] = value
            dislookup[c2, c1] = value
            return value
    def pairwiwseDistance(self, c1, c2, cElookup, aIlookup, dislookup):
        if (c1, c2) in dislookup:
            return dislookup[c1, c2]
        else:
            result = 0
            for a in c1:
                for b in c2:
                    result += self.clusterDistance(a, b, cElookup, aIlookup, dislookup)
            result /= float(len(c1) * len(c2))
            dislookup[c1, c2] = result
            dislookup[c2, c1] = result
            return result                 

    def buildTree(self):
        clusters = [(i, ) for i in range(self.input.D)]
        self.masks = [(i, ) for i in range(self.input.D)]
        random.shuffle(clusters)

        dislookup = {}
        aIlookup = {}
        cElookup = {}
        def allLowest():
            '''
            Internal function used to find the list of all clusters pairings
            with the current smallest distances.
            '''
            minDis = 3  # Max possible distance should be 2
            results = []
            for cluster1, cluster2 in combinations(clusters,2):
                dis = self.pairwiwseDistance(cluster1, cluster2, cElookup, aIlookup, dislookup )
                # dis = self.distance(cluster1, cluster2)
                if minDis > dis:
                    minDis = dis
                    results = [(cluster1, cluster2)]
                elif minDis == dis:
                    results.append((cluster1,cluster2))
            return results
        while len(clusters) > 1:
            c1, c2 = random.choice(allLowest())
            clusters.remove(c1)
            clusters.remove(c2)
            combined = c1 + c2
            clusters.append(combined)
            if len(clusters) != 1:
                self.masks.append(combined)
    
    def applyMask(self, p1, p2, mask):
        r = np.array([p2.randomKey[i] if i in mask else p1.randomKey[i]
                                        for i in range(self.input.D)])
        ind = Individual(self.dataPath, r)
        ind.eval()
        self.numEval+=1
        return ind
    def crossover(self, p1, p2):
        
        for mask in self.masks:
            child1 = self.applyMask(p1, p2, mask)
            child2 = self.applyMask(p2, p1, mask)    

            if min(p1.fitness, p2.fitness) > min(child1.fitness, child2.fitness):
                p1,p2 = child1, child2
        if p1.fitness < p2.fitness:
            return p1
        else:
            return p2
    
    def run(self):
        
        self.initPop()
        
        for i in range(self.numGeneration):
            self.evalPop()
            bestCost = self.getBestCost()
            self.buildTree()
            print(f"Epoch {i+1}/{self.numGeneration}: Best fitness: {bestCost} - NumEval: {self.numEval} ")

            # isIndivSelected = np.full(self.popSize, False)
            intermediatePop = []   

            while len(intermediatePop) < self.popSize:

                id1 = np.random.randint(self.popSize)
                id2 = id1
                while id2 == id1: id2 = np.random.randint(self.popSize)


                child = self.crossover(self.pop[id1], self.pop[id2])
                intermediatePop.append(child)
            
            self.pop = np.array(intermediatePop)




In [ ]:
# path = "../Data/idpc_10x5x425.idpc"
path = "../Data/idpc_10x10x1000.idpc"
# path = "../Data/idpc_15x15x3375.idpc"
# path = "../Data/idpc_15x30x12111.idpc"




problem = LinkageTreeAlgo(100, path, 200)
problem.run()